<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Graph_Convolutional_Network_(GCN)_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch_geometric

In [ ]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.optim as optim

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Example graph data structure
x = torch.tensor([[2, 1], [3, 1], [2, 4], [3, 4]], dtype=torch.float)  # Node features
edge_index = torch.tensor([[0, 1, 2, 3], [1, 0, 3, 2]], dtype=torch.long)  # Edges
y = torch.tensor([0, 1, 0, 1], dtype=torch.long)  # Labels
data = Data(x=x, edge_index=edge_index, y=y)

# Model, optimizer, and loss function setup
model = GCN(in_channels=2, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

print("Training completed!")

# Model evaluation
model.eval()
_, pred = model(data.x, data.edge_index).max(dim=1)
correct = (pred == data.y).sum().item()
accuracy = correct / data.num_nodes
print(f'Accuracy: {accuracy:.4f}')